5-7. 미니 프로젝트 : 가위바위보 분류기를 만들자 


<<목차>>

1)데이터를 준비하자

2)딥러닝 네트워크 설계하기

3)딥러닝 네트워크 학습시키기

4)얼마나 잘 만들었는지 확인하기(테스트)

5)더 좋은 네트워크 만들어보기

6)회고

1)데이터를 준비하자

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
import glob
import os
from tensorflow import keras
import matplotlib.pyplot as plt

print(tf.__version__)
print(np.__version__)

2022-10-07 23:50:55.687265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 23:50:55.845986: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 23:50:56.408353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.5/lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64
2022-10-07 23:50:56.408413: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not l

2.10.0
1.23.3


In [2]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
	
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/rock"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [3]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/scissor"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [4]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/paper"
resize_images(image_dir_path)

94  images to be resized.
94  images resized.


이미지들을 모두 28x28으로 리사이즈

In [52]:
import numpy as np

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1

    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

2)딥러닝 네트워크 설계하기

In [6]:
image_dir_path = "/home/june/다운로드/RCP"
(x_train, y_train)=load_data(image_dir_path)

x_train_norm = x_train/255.0

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 294 입니다.
x_train shape: (300, 28, 28, 3)
y_train shape: (300,)


In [34]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(256, (3, 3), activation = 'relu', input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(512, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(512, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation = 'relu'))
model.add(keras.layers.Dense(3, activation = 'softmax'))

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 256)       7168      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 11, 11, 512)       1180160   
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 5, 5, 512)        0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 3, 3, 512)         2359808   
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 1, 1, 512)       

3)딥러닝 네트워크 학습시키기

In [107]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train_norm, y_train, epochs = 10)

Epoch 1/10
10/10 [==============================] - 0s 3ms/step - loss: 1.2159 - accuracy: 0.3767
Epoch 2/10
10/10 [==============================] - 0s 4ms/step - loss: 1.1808 - accuracy: 0.3633
Epoch 3/10
10/10 [==============================] - 0s 4ms/step - loss: 1.1533 - accuracy: 0.3533
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 1.1341 - accuracy: 0.3767
Epoch 5/10
10/10 [==============================] - 0s 6ms/step - loss: 1.1207 - accuracy: 0.3167
Epoch 6/10
10/10 [==============================] - 0s 6ms/step - loss: 1.1019 - accuracy: 0.4067
Epoch 7/10
10/10 [==============================] - 0s 6ms/step - loss: 1.0875 - accuracy: 0.4700
Epoch 8/10
10/10 [==============================] - 0s 5ms/step - loss: 1.0676 - accuracy: 0.4200
Epoch 9/10
10/10 [==============================] - 0s 6ms/step - loss: 1.1080 - accuracy: 0.3567
Epoch 10/10
10/10 [==============================] - 0s 5ms/step - loss: 1.0861 - accuracy: 0.4500


4)얼마나 잘 만들었는지 확인하기(테스트)

In [108]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/test/rock"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [109]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/test/scissor"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [110]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
image_dir_path = "/home/june/다운로드/RCP/test/paper"
resize_images(image_dir_path)

100  images to be resized.
100  images resized.


In [111]:
image_dir_path = "/home/june/다운로드/RCP/test"
(x_test, y_test)=load_data(image_dir_path, 300)

x_test_norm = x_test/255.0

학습데이터(x_train)의 이미지 개수는 300 입니다.


In [148]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 48.3337 - accuracy: 0.3333 - 31ms/epoch - 3ms/step
test_loss: 48.333709716796875 
test_accuracy: 0.3333333432674408


정확도는 낮은 편이다. 다른 모델들을 사용해봐야 할것이다.

5)더 좋은 네트워크 만들어보기

In [141]:
n_channel_1=32
n_channel_2=32
n_dense=64
n_train_epoch=5

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [142]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 26, 26, 32)        896       
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 11, 11, 32)        9248      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 5, 5, 32)         0         
 g2D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 800)               0         
                                                                 
 dense_18 (Dense)            (None, 64)              

In [150]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

Epoch 1/10
10/10 [==============================] - 0s 3ms/step - loss: 16.9976 - accuracy: 0.4267
Epoch 2/10
10/10 [==============================] - 0s 4ms/step - loss: 3.1351 - accuracy: 0.5367
Epoch 3/10
10/10 [==============================] - 0s 4ms/step - loss: 3.9860 - accuracy: 0.4800
Epoch 4/10
10/10 [==============================] - 0s 4ms/step - loss: 1.7614 - accuracy: 0.5833
Epoch 5/10
10/10 [==============================] - 0s 4ms/step - loss: 0.8118 - accuracy: 0.7233
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7695 - accuracy: 0.7100
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - loss: 0.3327 - accuracy: 0.8600
Epoch 8/10
10/10 [==============================] - 0s 4ms/step - loss: 0.2382 - accuracy: 0.9367
Epoch 9/10
10/10 [==============================] - 0s 5ms/step - loss: 0.2459 - accuracy: 0.9233
Epoch 10/10
10/10 [==============================] - 0s 4ms/step - loss: 0.2938 - accuracy: 0.9000


In [151]:
image_dir_path = "/home/june/다운로드/RCP/test"
(x_test, y_test)=load_data(image_dir_path)

x_test_norm = x_test/255.0

학습데이터(x_train)의 이미지 개수는 300 입니다.


In [152]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 4.5788 - accuracy: 0.4167 - 112ms/epoch - 11ms/step
test_loss: 4.578839302062988 
test_accuracy: 0.4166666567325592


In [184]:
n_channel_1=8
n_channel_2=128
n_channel_3 = 256
n_dense=2048

IMG_HEIGHT = 28
IMG_WIDTH = 28

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', 
                              input_shape=(IMG_HEIGHT,IMG_WIDTH,3)))
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, kernel_regularizer=keras.regularizers.l2(0.001),activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

In [185]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 26, 26, 8)         224       
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 13, 13, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_40 (Conv2D)          (None, 11, 11, 128)       9344      
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 5, 5, 128)        0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 3, 3, 256)         295168    
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 1, 1, 256)      

In [186]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20)

Epoch 1/20
10/10 [==============================] - 0s 11ms/step - loss: 7.3194 - accuracy: 0.3800
Epoch 2/20
10/10 [==============================] - 0s 5ms/step - loss: 1.5159 - accuracy: 0.3200
Epoch 3/20
10/10 [==============================] - 0s 5ms/step - loss: 1.3917 - accuracy: 0.4000
Epoch 4/20
10/10 [==============================] - 0s 5ms/step - loss: 1.3244 - accuracy: 0.4967
Epoch 5/20
10/10 [==============================] - 0s 5ms/step - loss: 1.1735 - accuracy: 0.5700
Epoch 6/20
10/10 [==============================] - 0s 4ms/step - loss: 1.1007 - accuracy: 0.5933
Epoch 7/20
10/10 [==============================] - 0s 5ms/step - loss: 0.8959 - accuracy: 0.7633
Epoch 8/20
10/10 [==============================] - 0s 5ms/step - loss: 0.8586 - accuracy: 0.7300
Epoch 9/20
10/10 [==============================] - 0s 5ms/step - loss: 0.6847 - accuracy: 0.8400
Epoch 10/20
10/10 [==============================] - 0s 6ms/step - loss: 0.6146 - accuracy: 0.8433
Epoch 11/20
10/10 

In [187]:
image_dir_path = "/home/june/다운로드/RCP/test"
(x_test, y_test)=load_data(image_dir_path, 300)

x_test_norm = x_test/255.0

학습데이터(x_train)의 이미지 개수는 300 입니다.


In [188]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 6.3944 - accuracy: 0.5500 - 107ms/epoch - 11ms/step
test_loss: 6.3943681716918945 
test_accuracy: 0.550000011920929


값들을 더욱 증가시키면 루브릭 평가지표를 만족 시킬수 있을것 같다. 

In [349]:
n_channel_1=8
n_channel_2=128
n_channel_3 = 32
n_dense=512

IMG_HEIGHT = 28
IMG_WIDTH = 28

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', 
                              input_shape=(IMG_HEIGHT,IMG_WIDTH,3)))
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, kernel_regularizer=keras.regularizers.l2(0.001),activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_156 (Conv2D)         (None, 26, 26, 8)         224       
                                                                 
 max_pooling2d_156 (MaxPooli  (None, 13, 13, 8)        0         
 ng2D)                                                           
                                                                 
 conv2d_157 (Conv2D)         (None, 11, 11, 128)       9344      
                                                                 
 max_pooling2d_157 (MaxPooli  (None, 5, 5, 128)        0         
 ng2D)                                                           
                                                                 
 conv2d_158 (Conv2D)         (None, 3, 3, 32)          36896     
                                                                 
 max_pooling2d_158 (MaxPooli  (None, 1, 1, 32)       

In [350]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

Epoch 1/10
10/10 [==============================] - 1s 6ms/step - loss: 1.7278 - accuracy: 0.2933
Epoch 2/10
10/10 [==============================] - 0s 6ms/step - loss: 1.1508 - accuracy: 0.3367
Epoch 3/10
10/10 [==============================] - 0s 7ms/step - loss: 1.1168 - accuracy: 0.4733
Epoch 4/10
10/10 [==============================] - 0s 6ms/step - loss: 1.0542 - accuracy: 0.5467
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - loss: 0.9306 - accuracy: 0.5567
Epoch 6/10
10/10 [==============================] - 0s 4ms/step - loss: 0.8075 - accuracy: 0.6267
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7027 - accuracy: 0.7167
Epoch 8/10
10/10 [==============================] - 0s 6ms/step - loss: 0.5959 - accuracy: 0.7733
Epoch 9/10
10/10 [==============================] - 0s 4ms/step - loss: 0.5716 - accuracy: 0.7733
Epoch 10/10
10/10 [==============================] - 0s 5ms/step - loss: 0.4170 - accuracy: 0.8633


In [351]:
image_dir_path = "/home/june/다운로드/RCP/test"
(x_test, y_test)=load_data(image_dir_path)

x_test_norm = x_test/255.0

학습데이터(x_train)의 이미지 개수는 300 입니다.


In [352]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 3.0550 - accuracy: 0.3900 - 111ms/epoch - 11ms/step
test_loss: 3.055023431777954 
test_accuracy: 0.38999998569488525


6) 회고

- 이번 프로젝트에서 **어려웠던 점,**
    나의 environment 때문인지 텐서플로우 오류가 자주 나와 시간을 많이 까먹었다. cuda설정 하며 무언가 바뀐 것 같은데 진행이 안되곤 했다.


- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
    데이터가 좋지 않으면 처음부터 다시 해야 할 때가 있는 것 같다. 좋지 않은 데이터라면 내가 좋게 다듬는 방법을 알아야 할 것 같다. 데이터에 따라서 모델과 학습 량과 결과 등 모든 것이 달라질 수 있다.
    다른 모델은 오히려 층수가 더 적은데 정확도는 더 좋다. 많다고 무조건 좋은 것이 아닌 데이터나 상황에 따라서 알맞게 사용하는 것이 맞겠지만 아직 감이 생기려면 수천번 연습해야 할 것 같다.
    층 수 보다는 이미지의 특징, 알고리즘의 복잡도 등 더 다양한 요건들을 종합적으로 고려해야 하는 것 같다.

- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.
    모델의 층을 다르게 구성, 다양한 값들 또한 다르게 구성하여 테스트. 트레인을 많이 시켰더니 트레인 ACCURACY는 좋아지지만 테스트 ACCURACY는 낮아지는 경향성이 있다. 항상 오버피팅에 유의하여 학습시켜야 하는 것 같다.
    nDense 값이 너무 높아도 정확도가 떨어지고 너무 낮아도 정확도가 떨어졌다.


- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**.


- **자기 다짐**